In [18]:
## Library
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [20]:
#URL & Page
URL = "https://www.blockchain.com/btc/unconfirmed-transactions"
page = requests.get(URL)

#Make it readable
soup = bs(page.content)

#Find TIME, BTC-VALUE, USD-VALUE & HASH
results_numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
results_hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})

#Make arrays of only required values
only_time = [results_numbers[i] for i in range(0, len(results_numbers), 3)]
only_BTC = [results_numbers[i] for i in range(1, len(results_numbers), 3)]
only_USD = [results_numbers[i] for i in range(2, len(results_numbers), 3)]
only_hash = [results_hash[i] for i in range(0, len(results_hash), 1)]

#Make dataframe
dataF = {'Hash' : only_hash, 'Time' : only_time, 'BTC-value' : only_BTC, 'USD-value' : only_USD}
frame = pd.DataFrame(data=dataF)

#Print Dataframe
print(frame)

       Time          BTC-value      USD-value
0   [20:52]   [0.00213674 BTC]       [$80.93]
1   [20:52]   [0.00176902 BTC]       [$67.00]
2   [20:52]   [0.01416937 BTC]      [$536.67]
3   [20:52]   [0.35199494 BTC]   [$13,331.82]
4   [20:52]   [0.00703207 BTC]      [$266.34]
5   [20:52]   [0.04545899 BTC]    [$1,721.76]
6   [20:52]   [0.00396753 BTC]      [$150.27]
7   [20:52]   [4.11658673 BTC]  [$155,915.89]
8   [20:52]   [0.00289363 BTC]      [$109.60]
9   [20:52]   [0.00171801 BTC]       [$65.07]
10  [20:52]   [0.00941992 BTC]      [$356.78]
11  [20:52]   [0.16672360 BTC]    [$6,314.66]
12  [20:52]   [0.19669783 BTC]    [$7,449.94]
13  [20:52]   [0.00409578 BTC]      [$155.13]
14  [20:52]   [0.01236973 BTC]      [$468.50]
15  [20:52]   [0.00103460 BTC]       [$39.19]
16  [20:52]   [0.02831163 BTC]    [$1,072.30]
17  [20:52]   [0.20537565 BTC]    [$7,778.61]
18  [20:52]   [0.01593161 BTC]      [$603.41]
19  [20:52]   [0.00401512 BTC]      [$152.07]
20  [20:52]   [0.00269290 BTC]    